# Visualisation of convergence curves
installing new packages on metacebtrum: `/usr/bin/python3 -m pip install -U plotly`


In [1]:
import numpy as np
import plotly.graph_objects as go
import json

In [2]:
figure = go.Figure(
        layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("Energy convergence")))

In [12]:
# import the data from log file
OUT_NAME_suffixless="../models16_RBM_2_0.01_"
DS_AF_switch = 0 # 0...DS, 1...AF

exact_ground_energy = [-24,-34.0819077860][DS_AF_switch]
OUT_NAME_suffixless = OUT_NAME_suffixless+["DS_","AF_"][DS_AF_switch]
figure.add_hline(y=exact_ground_energy, opacity=1, line_width=1)
no_of_runs = 2
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+["normal","MSR"][i]+".log")))
names = ["normal","MSR"]
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-use_MSR)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-use_MSR)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

### N=16 hyperparameter search

In [4]:
j1 = 0.9
OUT_NAME_suffixless="N=16_search/"
no_of_runs = 5
exact_energies = {0.1: -24, 0.3: -24, 0.5: -24, 0.7: -25.4010078362, 0.9:-34.0819077860}

figure = go.Figure(
        layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("Energy convergence")))
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i+1)+"/RBM16_"+str(j1)+"_0.log")))
names = ["η=0.010,α=16","η=0.010,α=08","S=0,η=0.01,α=16","η=0.002,α=16","η=0.100,α=16",]

if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(x=data[i]["Energy"]["iters"], y=energy_convergence[i],name=names[i]))
figure.add_trace(go.Scatter(x=(0,3000),y=(exact_energies[j1],exact_energies[j1]),mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy"))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy",title="Energy convergence for J1="+str(j1))
figure.show()

## Ressurection of the average final accuracy

Kopírovací skript
```
scp logs/*.log mezic@tarkil8:~/diplomka/netket_scripts/logs
scp logs/*.mpack mezic@tarkil8:~/diplomka/netket_scripts/logs/models
```

In [5]:
OUT_NAME = "logs/models16"
names = ["RBM_2","RBM_16","RBM_2notVisible","RBM_16notVisible","RBMSymm_16aut","RBMSymm_128aut","RBMSymm_16trans","RBMSymm_128trans","GCNN_my_32","GCNN_my_8","GCNN_my_8trans","GCNN_my_8notVisible","GCNN_aut[8,4]","GCNN_trans[8,4]","GCNN_aut[16,16]"]
ETAS = [0.01,0.05,0.2]
INDICES = range(4,8)
no_etas = len(ETAS)
no_of_runs = 4 # types of runs: DS_norm, DS_MSR, AF_norm, AF_MSR
no_repeats = 4 # repeats of the same run due to inconsistent results
exact_ground_energies = np.array([-24,-24,-34.0819077860,-34.0819077860])
threshold_energies = 0.995*exact_ground_energies

for m in INDICES:
    steps_until_conv = np.zeros((no_repeats,no_of_runs))
    min_energy_error = np.zeros((no_repeats,no_of_runs))
    average_final_energy = np.zeros((no_repeats,no_of_runs))
    ETA = ETAS[m%no_etas]
    name = names[m//no_etas]
    for j in range(no_repeats):
        data = []
        MODEL_LOG_NAME = OUT_NAME+"_"+name+"_"+str(ETAS[m%no_etas])+"_"+str(j)+"_"
        for i in range(no_of_runs):
            data.append(json.load(open(MODEL_LOG_NAME+["DS_normal","DS_MSR","AF_normal","AF_MSR"][i]+".log")))
        if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
            energy_convergence = np.asarray([data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)])
        else:
            energy_convergence = np.asarray([data[i]["Energy"]["Mean"] for i in range(no_of_runs)])
        
        steps_until_conv[j] = np.asarray([next((i for i,v in enumerate(energy_convergence[j]) if v < threshold_energies[j]), np.inf) for j in range(no_of_runs)])
        min_energy_error[j] = [np.min(np.abs(energy_convergence[j] - exact_ground_energies[j])) for j in range(no_of_runs)]
        average_final_energy[j] = np.average(energy_convergence[:,-50:],axis=1)
    
    min_steps = np.min(steps_until_conv,axis=0)
    min_min_energy_error = np.min(min_energy_error,axis=0)
    min_avg_final_energy_relative_error = -np.log10((exact_ground_energies-np.min(average_final_energy,axis=0))/exact_ground_energies)
    print('{:2.0f} {:<35}   {:4.3f}  {:5.0f} {:5.2f} {:8.3}   {:5.0f} {:5.2f} {:8.3f}   {:5.0f} {:5.2f} {:8.3}  {:5.0f} {:5.2f} {:8.3f}'.format(m, OUT_NAME+"_"+name+"_"+str(ETAS[m%no_etas]),ETAS[m%no_etas],min_steps[0],min_avg_final_energy_relative_error[0],min_min_energy_error[0],min_steps[1],min_avg_final_energy_relative_error[1],min_min_energy_error[1],min_steps[2],min_avg_final_energy_relative_error[2],min_min_energy_error[2],min_steps[3],min_avg_final_energy_relative_error[3],min_min_energy_error[3]))

 4 logs/models16_RBM_16_0.05             0.050     28  4.55 1.38e-08      52  4.87    0.000     inf  1.29      1.7     65  2.96    0.000
 5 logs/models16_RBM_16_0.2              0.200    inf  0.23     14.0     inf  0.23   13.995     inf  0.20     18.9    inf  0.41   13.104
 6 logs/models16_RBM_2notVisible_0.01    0.010    153  3.88 5.12e-09     195  4.02    0.000     292  2.28   0.0755    345  2.30    0.072
 7 logs/models16_RBM_2notVisible_0.05    0.050     33  4.61 1.52e-07      46  4.74    0.000      84  2.29   0.0747     67  2.33    0.039


In [8]:
figure = go.Figure(
        layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("Energy convergence")))

### Convergence plotting of the benchmarked models

In [12]:
# import the data from log file
OUT_NAME_suffixless="logs/models16_RBM_2_0.05_0_"
DS_AF_switch = 0 # 0...DS, 1...AF

exact_ground_energy = [-24,-34.0819077860][DS_AF_switch]
OUT_NAME_suffixless = OUT_NAME_suffixless+["DS_","AF_"][DS_AF_switch]
figure.add_hline(y=exact_ground_energy, opacity=1, line_width=1)
no_of_runs = 2
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+["normal","MSR"][i]+".log")))
names = ["normal","MSR"]
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-use_MSR)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-use_MSR)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

In [14]:
np.average(np.asarray(energy_convergence)[1,-50:])

-23.99710146187375